In [2]:
import polars as pl
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris

from datetime import datetime

# series

Una serie è una sequenza unidimensionale di valori dello stesso tipo.

In [1]:
# una serie di numeri interi
s = pl.Series("series", [1, 2, 3, 4], dtype=pl.Int64)
print(s)

NameError: name 'pl' is not defined

# dataframe

Un dataframe è una tabella bidimensionale di dati, in cui ogni colonna può contenere valori di tipo diverso.

In [3]:
df = pl.DataFrame(
    {
        "integer": [1, 2, 3, 4, 5],
        "date": [
            datetime(2022, 1, 1),
            datetime(2022, 1, 2),
            datetime(2022, 1, 3),
            datetime(2022, 1, 4),
            datetime(2022, 1, 5),
        ],
        "float": [4.0, 5.0, 6.0, 7.0, 8.0],
    }
)

print(df)

shape: (5, 3)
┌─────────┬─────────────────────┬───────┐
│ integer ┆ date                ┆ float │
│ ---     ┆ ---                 ┆ ---   │
│ i64     ┆ datetime[μs]        ┆ f64   │
╞═════════╪═════════════════════╪═══════╡
│ 1       ┆ 2022-01-01 00:00:00 ┆ 4.0   │
│ 2       ┆ 2022-01-02 00:00:00 ┆ 5.0   │
│ 3       ┆ 2022-01-03 00:00:00 ┆ 6.0   │
│ 4       ┆ 2022-01-04 00:00:00 ┆ 7.0   │
│ 5       ┆ 2022-01-05 00:00:00 ┆ 8.0   │
└─────────┴─────────────────────┴───────┘


In [4]:
df = pl.DataFrame(
    {
        "nrs": [1, 2, 3, None, 5],
        "names": ["foo", "ham", "spam", "egg", None],
        "random": np.random.rand(5),
        "groups": ["A", "A", "B", "C", "B"],
    }
)
print(df)

shape: (5, 4)
┌──────┬───────┬──────────┬────────┐
│ nrs  ┆ names ┆ random   ┆ groups │
│ ---  ┆ ---   ┆ ---      ┆ ---    │
│ i64  ┆ str   ┆ f64      ┆ str    │
╞══════╪═══════╪══════════╪════════╡
│ 1    ┆ foo   ┆ 0.245004 ┆ A      │
│ 2    ┆ ham   ┆ 0.89495  ┆ A      │
│ 3    ┆ spam  ┆ 0.680767 ┆ B      │
│ null ┆ egg   ┆ 0.800706 ┆ C      │
│ 5    ┆ null  ┆ 0.155166 ┆ B      │
└──────┴───────┴──────────┴────────┘


# Contesti

## select

la funzione `with_columns` è simile alla select: La differenza è che con la `with_columns` le colonne originali rimangono, quelle nuove vengono aggiunte.


In [22]:
# Applica delle espressioni alle colonne
df.select(
    pl.col("names").sort(),
    pl.col("random").cum_sum().alias("random_cumsum")
)

names,random_cumsum
str,f64
null,0.465309
"""egg""",1.199363
"""foo""",1.541654
"""ham""",1.859295
"""spam""",2.199634


## filter

Seleziona le righe che soddisfano una condizione.

In [23]:
df.filter(pl.col("random") < 0.5)

nrs,names,random,groups
i64,str,f64,str
1,"""foo""",0.465309,"""A"""
3,"""spam""",0.342291,"""B"""
null,"""egg""",0.317641,"""C"""
5,null,0.340339,"""B"""


<br>Si possono concatenare più filtri con l'operatore `&` (and) o `|` (or).

In [30]:
df.filter((pl.col("random") < 0.7) & (pl.col("random") > 0.32))

nrs,names,random,groups
i64,str,f64,str
1,"""foo""",0.465309,"""A"""
3,"""spam""",0.342291,"""B"""
5,null,0.340339,"""B"""


Passare più filtri come argomenti è equivalente a concatenarli con l'operatore `&`.

In [31]:
df.filter(pl.col("random") < 0.7, pl.col("random") > 0.32)

nrs,names,random,groups
i64,str,f64,str
1,"""foo""",0.465309,"""A"""
3,"""spam""",0.342291,"""B"""
5,null,0.340339,"""B"""


<br>Si possono specificare poi una serie di uguaglianze che devono essere soddisfatte con dei parametri che
hanno i nomi delle colonne

In [25]:
df.filter(pl.col("random") < 0.5, groups="B")

nrs,names,random,groups
i64,str,f64,str
3,"""spam""",0.342291,"""B"""
5,null,0.340339,"""B"""


## groupby

Raggrouppa i dati in base ai valori di una o più colonne e poi applica una o più funzioni di aggregazione.

In [7]:
df.group_by("groups").agg(
    pl.sum("nrs"),  # sum nrs by groups
    pl.col("random").count().alias("count"),  # count group members
    # sum random where name != null
    pl.col("random").filter(pl.col("names").is_not_null()).sum().name.suffix("_sum"),
    pl.col("names").reverse().alias("reversed names"),
)

groups,nrs,count,random_sum,reversed names
str,i64,u32,f64,list[str]
"""B""",8,2,0.708213,"[null, ""spam""]"
"""A""",3,2,0.864429,"[""ham"", ""foo""]"
"""C""",0,1,0.636089,"[""egg""]"


# Espressioni

Un'espressione restituisce una serie.

In [8]:
# una semplice espressione che rappresenta una colonna
exp1 = pl.col("a string column")

type(exp1)

polars.expr.expr.Expr

In [9]:
# una semplice espressione che rappresenta una costante
exp2 = pl.lit("_suffix")

type(exp2)

polars.expr.expr.Expr

In [10]:
# le espessioni possono essere combinate. exp3 è una nuova espressione
# che rappresenta la concatenazione di exp1 e exp2
exp3 = exp1 + exp2

In [11]:
# concatenazione di più espessioni
# 1. seleziona la colonna "a column"
# 2. filtra i valori maggiori di 5
# 3. ordina in ordine decrescente
# 4. assegna il risultato a "output column"
exp = (
    pl.col("a column").
    filter(pl.col("a column") > 5).
    sort(descending=True).
    alias("output column")
)

type(exp)

polars.expr.expr.Expr

# Lazy vs Eager

In [12]:
# creo il csv Iris

df_iris = load_iris(as_frame=True)["frame"]

# rename columns
df_iris.columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]

df_iris.to_csv("data/iris.csv")


## Eager

In [13]:
# legge il file csv
df = pl.read_csv("data/iris.csv")
# solo i record con sepal_length > 5
df_small = df.filter(pl.col("sepal_length") > 5)
# raggruppa per specie e calcola la larghezza media del sepalo per ogni specie
df_agg = df_small.group_by("species").agg(pl.col("sepal_width").mean())
print(df_agg)

shape: (3, 2)
┌─────────┬─────────────┐
│ species ┆ sepal_width │
│ ---     ┆ ---         │
│ i64     ┆ f64         │
╞═════════╪═════════════╡
│ 2       ┆ 2.983673    │
│ 0       ┆ 3.713636    │
│ 1       ┆ 2.804255    │
└─────────┴─────────────┘


## Lazy

Una query lazy può iniziare con
1. la lettura di un file con una funzione del tipo `scan_*`
2. la trasformazione di un Dataframe in un LazyFrame con il metodo `lazy`

Alla fine di una query lazy si ottiene il risultato con il metodo `collect`.
<br> Alternativamente, a scopo di debug, si può usare il metodo `fetch` che legge dalla sorgente solo le prime $n$ righe.

[Qui](https://docs.pola.rs/user-guide/lazy/optimizations/) vengono spiegate le ottimizzazioni che vengono fatte quando si lavora con query lazy.

In [14]:
q = (
    pl.scan_csv("data/iris.csv")
    .filter(pl.col("sepal_length") > 5)
    .group_by("species")
    .agg(pl.col("sepal_width").mean())
)

df = q.collect()
print(df)

shape: (3, 2)
┌─────────┬─────────────┐
│ species ┆ sepal_width │
│ ---     ┆ ---         │
│ i64     ┆ f64         │
╞═════════╪═════════════╡
│ 1       ┆ 2.804255    │
│ 2       ┆ 2.983673    │
│ 0       ┆ 3.713636    │
└─────────┴─────────────┘


# Streaming API

In [16]:
q = (
    pl.scan_csv("data/iris.csv")
    .filter(pl.col("sepal_length") > 5)
    .group_by("species")
    .agg(pl.col("sepal_width").mean())
)

df = q.collect(streaming=True)
print(df)

shape: (3, 2)
┌─────────┬─────────────┐
│ species ┆ sepal_width │
│ ---     ┆ ---         │
│ i64     ┆ f64         │
╞═════════╪═════════════╡
│ 1       ┆ 2.804255    │
│ 0       ┆ 3.713636    │
│ 2       ┆ 2.983673    │
└─────────┴─────────────┘


In [ ]:
print(q.explain(streaming=True))

--- STREAMING
AGGREGATE
	[col("sepal_width").mean()] BY [col("species")] FROM

    Csv SCAN iris.csv
    PROJECT 3/6 COLUMNS
    SELECTION: [(col("sepal_length")) > (5.0)]  --- END STREAMING

  DF []; PROJECT */0 COLUMNS; SELECTION: "None"


<ipython-input-47-a08fcb332bbd>:1: UserWarning: Cannot combine 'streaming' with 'comm_subplan_elim'. CSE will be turned off.
  print(q.explain(streaming=True))


# Trasformazioni

## Join

## Concatenzione

## Pivot

## Melts

## Time series

# Valori mancanti

In polars i valori mancanti sono rappresentati da null (non da NaN). Nei tipi numerici un valore può essere NaN, ma polars non lo considera un valore mancante.
<br>Diveso comportamento rispetto tra null e NaN:
<br>se si fa la media di una colonna con valori NaN, il risultato sarà NaN, mentre se si fa la media di una colonna con valori null, il risultato sarà la media dei valori non nulli.

Funzioni per gestire i valori mancanti:
<br>
`is_null()`: restituisce un booleano che indica se il valore è mancante
<br>
`null_count()`: restituisce il numero di valori mancanti
<br>
`fill_null()`: sostituisce i valori mancanti con un valore specificato
<br>
`is_nan()`: restituisce un booleano che indica se il valore è NaN
<br>
`fill_nan()`: sostituisce i valori NaN con un valore specificato